<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

### Import the API

Run the following cell to import the API into your session.

In [ ]:
# To use the Google Collab T4 GPU:
#import tensorflow as tf
# Check GPU availability
#print("GPU Available:", tf.config.experimental.list_physical_devices("GPU"))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import ee
import geopandas
import geemap
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import Point
import numpy as np
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [19]:
# function to sample 3DEP data using gdf points
def sample_3dep_data(gdf):
    # Load the 3DEP DEM ImageCollection
    dem_collection = ee.ImageCollection("USGS/3DEP/1m")

    # Convert the GeoDataFrame to an Earth Engine FeatureCollection
    ee_points = ee.FeatureCollection(gdf)

    # Define a function to sample elevation, latitude, and longitude values at the points
    def sample_at_points(image):
        sampled_points = image.sample(
            region=ee_points.geometry(),
            scale=1,  # 1-meter scale for 1-meter DEM data
        )
        return sampled_points

    # Map the sampling function over the ImageCollection
    sampled_points_collection = dem_collection.map(sample_at_points)

    # Flatten the resulting FeatureCollection
    sampled_points_flat = sampled_points_collection.flatten()

    # Define a function to extract elevation, latitude, and longitude values
    def extract_3dep_data(feature):
        elevation = feature.get("elevation")
        return feature.set({
            "elevation": elevation  # Assign elevation to the 'altitude' attribute
        })

    # Apply the extraction function to each sampled point
    extracted_data = sampled_points_flat.map(extract_3dep_data)

    # Convert the result back to a GeoDataFrame
    extracted_gdf = gpd.GeoDataFrame.from_features(extracted_data.getInfo())

    return extracted_gdf

In [20]:
from branca.utilities import none_min
# Define the bounding box coordinates in lat/long
min_lon, min_lat = -121.27390, 36.69001
max_lon, max_lat = -121.24352, 36.71644

# point sampling interval
interval_meters = 300

# Calculate the latitude and longitude ranges
lon_range = max_lon - min_lon
lat_range = max_lat - min_lat

num_points_x = int((max_lon - min_lon) * (111111.32 / interval_meters))  # Ensure it's positive
num_points_y = int((max_lat - min_lat) * (111111.32 / interval_meters)) # Ensure it's positive

# Adjust longitude calculation to handle negative longitudes
if min_lon < 0:
    min_lon = 360 + min_lon
    max_lon = 360 + max_lon
    negative_longitudes = True
    print(negative_longitudes)

print(min_lon, min_lat)
print(max_lon, max_lat)

num_points_x = int((max_lon - min_lon) * (111111.32 / interval_meters))
num_points_y = int((max_lat - min_lat) * (111111.32 / interval_meters))

# Generate arrays of longitude and latitude values
lon_values = np.linspace(min_lon, max_lon, num_points_x)
lat_values = np.linspace(min_lat, max_lat, num_points_y)
print(lon_values)

# Adjust longitudes back to the correct range if necessary
if negative_longitudes is True:
  lon_values = np.where(lon_values, lon_values - 360, lon_values)
  print(lon_values)

# Create a grid of points using meshgrid
lon_grid, lat_grid = np.meshgrid(lon_values, lat_values)

# Flatten the grids into 1D arrays
lon_flat = lon_grid.flatten()
lat_flat = lat_grid.flatten()

# Create a GeoDataFrame from the points
geometry = [Point(lon, lat) for lon, lat in zip(lon_flat, lat_flat)]
points_gdf = gpd.GeoDataFrame(geometry=geometry)

# Extract numeric values from lon_flat and lat_flat
longitudes = lon_flat.tolist()
latitudes = lat_flat.tolist()

## Create attributes for lat,long,altitude
# Initialize empty columns
points_gdf['latitude'] = latitudes
points_gdf['longitude'] = longitudes
points_gdf['elevation'] = None

print(points_gdf)

print("Total number of sampling points: ",len(points_gdf))
if len(points_gdf)>5000:
  print("Warning, number of points exceeds request limit (5,000)")
  print("Please Choose a smaller AOI or larger sampling interval before continuing")

True
238.7261 36.69001
238.75648 36.71644
[238.7261   238.729138 238.732176 238.735214 238.738252 238.74129
 238.744328 238.747366 238.750404 238.753442 238.75648 ]
[-121.2739   -121.270862 -121.267824 -121.264786 -121.261748 -121.25871
 -121.255672 -121.252634 -121.249596 -121.246558 -121.24352 ]
                       geometry  latitude   longitude elevation
0   POINT (-121.27390 36.69001)  36.69001 -121.273900      None
1   POINT (-121.27086 36.69001)  36.69001 -121.270862      None
2   POINT (-121.26782 36.69001)  36.69001 -121.267824      None
3   POINT (-121.26479 36.69001)  36.69001 -121.264786      None
4   POINT (-121.26175 36.69001)  36.69001 -121.261748      None
..                          ...       ...         ...       ...
94  POINT (-121.25567 36.71644)  36.71644 -121.255672      None
95  POINT (-121.25263 36.71644)  36.71644 -121.252634      None
96  POINT (-121.24960 36.71644)  36.71644 -121.249596      None
97  POINT (-121.24656 36.71644)  36.71644 -121.246558      No

In [21]:
points_gdf.crs = "EPSG:4326"  # WGS84

# Convert GeoDataFrame to Earth Engine objects
ee_points = geemap.geopandas_to_ee(points_gdf)

result_gdf = sample_3dep_data(ee_points)

In [22]:
# View the sampled elevation points
# Conversion from gdf to ee object lost the geometry, lat, long columns
print(result_gdf)
print(type(result_gdf))

   geometry   elevation
0      None  351.222595
1      None  305.486145
2      None  323.393433
3      None  324.583557
4      None  310.290283
..      ...         ...
94     None  239.964767
95     None  300.238403
96     None  282.850250
97     None  268.594360
98     None  223.753098

[99 rows x 2 columns]
<class 'geopandas.geodataframe.GeoDataFrame'>


In [23]:
# Add Longitude & Latitude Columns to Result GDF
result_gdf['geometry'] = points_gdf['geometry']
result_gdf['latitude'] = points_gdf['latitude']
result_gdf['longitude'] = points_gdf['longitude']
result_gdf

,geometry,elevation,latitude,longitude
0,POINT (-121.27390 36.69001),351.222595,36.69001,-121.273900
1,POINT (-121.27086 36.69001),305.486145,36.69001,-121.270862
2,POINT (-121.26782 36.69001),323.393433,36.69001,-121.267824
3,POINT (-121.26479 36.69001),324.583557,36.69001,-121.264786
4,POINT (-121.26175 36.69001),310.290283,36.69001,-121.261748
...,...,...,...,...
94,POINT (-121.25567 36.71644),239.964767,36.71644,-121.255672
95,POINT (-121.25263 36.71644),300.238403,36.71644,-121.252634
96,POINT (-121.24960 36.71644),282.850250,36.71644,-121.249596
97,POINT (-121.24656 36.71644),268.594360,36.71644,-121.246558


In [ ]:
# Export Sampled points as a shapefile
